## 데이터마이닝 프로젝트

In [100]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score, calinski_harabasz_score, davies_bouldin_score

import warnings
warnings.filterwarnings('ignore')


In [101]:
# 데이터 로드

FP_PATH    = "floating_population.csv"   # 유동인구
SALES_PATH = "sales.csv"                 # 매출
STORE_PATH = "store.csv"                 # 점포


In [ ]:
def read_korean_csv(path: str) -> pd.DataFrame:
    """Try common Korean encodings."""
    for enc in ("utf-8", "cp949", "euc-kr"):
        try:
            return pd.read_csv(path, encoding=enc)
        except UnicodeDecodeError:
            continue
    return pd.read_csv(path)

fp    = read_korean_csv(FP_PATH)
fp = fp[fp["기준_년분기_코드"] // 10 == 2024]
sales = read_korean_csv(SALES_PATH)
store = read_korean_csv(STORE_PATH)

print(fp.shape, sales.shape, store.shape)


In [ ]:
f = lambda x: display(x["기준_년분기_코드"].unique())
f(fp)
f(sales)
f(store)

In [ ]:
def get_columns(df):
    display(df.columns)
get_columns(fp)
get_columns(sales)
get_columns(store)

In [ ]:
#데이터 전처리
def make_numeric(df, drop_cols):
    return [c for c in df.columns if c not in drop_cols and df[c].dtype != 'object']

sales_num = make_numeric(sales, ["행정동_코드", "기준_년분기_코드"])
store_num = make_numeric(store, ["행정동_코드", "기준_년분기_코드"])

sales_agg = (sales
             .groupby(["기준_년분기_코드", "행정동_코드"])[sales_num]
             .sum()
             .reset_index())

store_agg = (store
             .groupby(["기준_년분기_코드", "행정동_코드"])[store_num]
             .median() # 0 값이 많아 median과 차이가 있는데 어떤 걸로 할지?
             .reset_index())

# merged 건들면 안됨!
merged = (fp
          .merge(sales_agg, on=["기준_년분기_코드", "행정동_코드"], how="left")
          .merge(store_agg, on=["기준_년분기_코드", "행정동_코드"], how="left"))
merged = merged.dropna(axis=0)
print("Merged:", merged.shape)


In [ ]:
merged.isna().sum().sum()

In [ ]:
# 표준화 후 PCA 분석
feature_cols = [c for c in merged.columns
                if c not in ("행정동_코드", "기준_년분기_코드") and merged[c].dtype != 'object']

scaler = StandardScaler()
X_scaled = scaler.fit_transform(merged[feature_cols])

pca = PCA(n_components=0.95, random_state=42, svd_solver='full')
X_pca = pca.fit_transform(X_scaled)

print(f"PCA → {X_pca.shape[1]} components (cum var {pca.explained_variance_ratio_.sum():.2%})")


In [ ]:
print(f"PCA → {X_pca.shape[1]} components (each var \n{pca.explained_variance_ratio_})")

-> 8개 주성분으로 원본 데이터의 분산 95.05% 설명 가능

In [109]:
loadings = pd.DataFrame(
    pca.components_.T,                         # (변수 개수 × 주성분 개수) 행렬
    index=feature_cols,                        # 행 = 원본 변수 이름
    columns=[f"PC{i+1}" for i in range(pca.n_components_)]
)

In [ ]:
len(merged.columns)

In [ ]:
for pc in loadings.columns:          # 'PC1' … 'PC7'
    top = (loadings[pc]
           .sort_values(ascending=False, key=abs)
           .head(30))                # 상위 8개 변수
    print(f"\n{pc} 상위 기여 변수:")
    print(top)

In [ ]:
# kmeans Clustering 사용하기 위해 k 설정하는 과정(지표 비교)

def eval_k(X, k):
    km = KMeans(n_clusters=k, random_state=42, n_init=10).fit(X)
    labels = km.labels_
    return {
        "k": k,
        "inertia": km.inertia_,
        "silhouette": silhouette_score(X, labels, sample_size=min(5000, len(X)), random_state=42),
        "calinski": calinski_harabasz_score(X, labels),
        "davies": davies_bouldin_score(X, labels)
    }

scores = pd.DataFrame([eval_k(X_pca, k) for k in range(2, 11)])
display(scores.set_index("k").round(2))

best_k = scores.loc[scores['silhouette'].idxmax(), 'k']
print("Best k (silhouette):", best_k)


In [ ]:
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt

ks, inertias = [], []
for k in range(2, 11):
    km = KMeans(n_clusters=k, random_state=42, n_init=10).fit(X_pca)
    ks.append(k)
    inertias.append(km.inertia_)

plt.plot(ks, inertias, 'o-')
plt.title("Elbow Method")
plt.xlabel("k"); plt.ylabel("Inertia")
plt.show()

In [114]:
### silhouette score는 k=2에서 제일 높았지만, 2개의 클러스터는 적다고 느껴지기도 하고 엘보우 메소드를 참고해서 k=4로 해봤습니다!

In [ ]:
# kmeans 클러스터링 적용
kmeans = KMeans(n_clusters=4, random_state=42, n_init=10)
merged['cluster'] = kmeans.fit_predict(X_pca)
merged['cluster'].value_counts().sort_index()


In [ ]:
# 시각화
plt.figure(figsize=(5,4))
plt.scatter(X_pca[:,0], X_pca[:,1], c=merged['cluster'], s=8, cmap='tab10')
plt.title('Clusters on PC1 vs PC2')
plt.show()  #PCA 1·2 축 산점도

plt.figure(figsize=(4,3))
merged['cluster'].value_counts().sort_index().plot(kind='bar')
plt.title('Cluster sizes')
plt.show() #cluster 별 크기

pc_df = pd.DataFrame(X_pca, columns=[f"PC{i+1}" for i in range(X_pca.shape[1])])
pc_df["cluster"] = merged["cluster"]
mean_matrix = pc_df.groupby("cluster").mean()

plt.figure(figsize=(6,3))
plt.imshow(mean_matrix, aspect="auto"); plt.colorbar()
plt.yticks(range(len(mean_matrix)), mean_matrix.index)
plt.title("Cluster Mean by PC"); plt.xlabel("PC"); plt.ylabel("Cluster")
plt.tight_layout(); plt.show() #군집별 PC 평균 Heatmap



In [ ]:
from sklearn.manifold import TSNE

for i in [10, 30, 50, 70]:
    tsne_model = TSNE(
        n_components=2,      # 축소할 차원 (보통 2 또는 3)
        perplexity=i,     # 가장 중요한 파라미터 중 하나. 데이터 포인트 주변의 로컬 이웃 수와 관련. (5 ~ 50 사이 값 권장)
        learning_rate='auto',# 최적화 과정에서의 학습률. 'auto' (scikit-learn 1.2 이상) 또는 10 ~ 1000 사이 값.
                            # 이전 버전에서는 learning_rate=200.0 이 기본값이었음.
        n_iter=1000,         # 최적화를 위한 반복 횟수 (최소 250, 보통 1000 이상 권장)
        init='pca',          # 초기 임베딩 방법 ('random' 또는 'pca'). 'pca'가 더 안정적이고 좋은 결과를 주는 경향이 있음.
        random_state=42      # 결과 재현을 위한 시드 값
    )

    # t-SNE 적용 (원본 X 또는 X_scaled 사용)
    tsne_results = tsne_model.fit_transform(X_scaled) # 또는 X_scaled

    # 2D 시각화
    plt.figure(figsize=(10, 7))
    scatter = plt.scatter(tsne_results[:, 0], tsne_results[:, 1], c=merged["cluster"], cmap='viridis', alpha=0.7)


    plt.title('t-SNE visualization of data')
    plt.xlabel('t-SNE Component 1')
    plt.ylabel('t-SNE Component 2')
    plt.grid(True)
    plt.show()

In [ ]:
#PC 평균
display(
    mean_matrix.round(2)
               .style.background_gradient(cmap="Blues", axis=None)
               .set_caption("클러스터별 주성분 평균")
)
#원본 변수 — 변별력 상위 8개
cluster_avg = merged.groupby("cluster")[feature_cols].mean()
top_vars = cluster_avg.var().sort_values(ascending=False).head(8).index

display(
    cluster_avg[top_vars].T.round(1)
               .style.background_gradient(cmap="YlOrRd", axis=1)
               .set_caption("클러스터별 평균 — 변별력 상위 8개 변수")
)

In [119]:
# merged에서 일부 열만 추출한 걸로 클러스터링 해보기
# 
new_merged = merged[['점포_수','프랜차이즈_점포_수',"개업_율","폐업_률","행정동_코드","당월_매출_건수", "당월_매출_금액",'시간대_06~11_매출_금액', '시간대_11~14_매출_금액','시간대_14~17_매출_금액', '시간대_17~21_매출_금액', '시간대_21~24_매출_금액']]
new_merged['여성_비율'] = merged['여성_매출_건수'] / merged['남성_매출_건수']
new_merged['20대_매출_금액_비율'] = merged['연령대_20_매출_금액'] / merged['당월_매출_금액']
new_merged['20대_유동인구_비율'] = merged['연령대_20_유동인구_수'] / (merged['연령대_50_유동인구_수']+merged['연령대_20_유동인구_수']+merged['연령대_30_유동인구_수']+merged['연령대_40_유동인구_수']+merged['연령대_60_이상_유동인구_수'])



In [ ]:

# 새로운 데이터셋에 대한 PCA 수행
scaler = StandardScaler()
X_scaled = scaler.fit_transform(new_merged)

# 2개짜리 pca
pca2 = PCA(n_components=2)
X_pca2 = pca2.fit_transform(X_scaled)

pca = PCA(n_components=3)  # 2개의 주성분만 사용
X_pca = pca.fit_transform(X_scaled)

print(pca.components_)
print(pca.n_components_)


In [ ]:
# 클러스터링 및 2차원 시각화

kmeans = KMeans(n_clusters=5, random_state=42, n_init=10)
new_merged['cluster'] = kmeans.fit_predict(X_pca)

# 2차원 시각화
plt.figure(figsize=(10, 8))
plt.scatter(X_pca[:,0], X_pca[:,1], c=new_merged['cluster'], cmap='viridis', alpha=0.3)
plt.xlabel('PC1')
plt.ylabel('PC2')
plt.show()


In [ ]:
# 클러스터링 적용
kmeans = KMeans(n_clusters=5, random_state=42, n_init=10)
new_merged['cluster'] = kmeans.fit_predict(X_pca)

# 3차원 시각화
from mpl_toolkits.mplot3d import Axes3D

fig = plt.figure(figsize=(10, 8))
ax = fig.add_subplot(111, projection='3d')

ax.scatter(X_pca[:,0], X_pca[:,1], X_pca[:,2], c=new_merged['cluster'], cmap='viridis', alpha=0.3)

ax.set_xlabel('PC1')
ax.set_ylabel('PC2')
ax.set_zlabel('PC3')
plt.show()


In [ ]:

# DBSCAN으로 클러스터링 해보기
from sklearn.cluster import DBSCAN


# DBSCAN 적용
dbscan = DBSCAN(eps=0.1, min_samples=10, metric='euclidean')
new_merged['cluster'] = dbscan.fit_predict(X_pca)

# 3d 시각화

from mpl_toolkits.mplot3d import Axes3D

fig = plt.figure(figsize=(10, 8))
ax = fig.add_subplot(111, projection='3d')

ax.scatter(X_pca[:,0], X_pca[:,1], X_pca[:,2], c=new_merged['cluster'], cmap='viridis', alpha=0.1)

ax.set_xlabel('PC1')
ax.set_ylabel('PC2')
ax.set_zlabel('PC3')
plt.show()



